In [31]:

question = "How can I mark a trace as successful or failed?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

<think>
Okay, so the user is asking how to mark a trace as successful or failed in LangChain using LangSmith. I remember that the context provided talks about adding metadata and tags using the 'langsmith_extra' parameter. 

Hmm, the context doesn't mention anything specific about marking traces as successful or failed. It only explains how to add metadata and tags, like setting a 'metadata' key or adding tags to the dictionary. So, maybe the user is looking for a way to categorize their traces based on execution results.

I think the best approach is to suggest using tags for this purpose. Since tags are used to categorize and filter traces, they could add a tag like 'success' or 'failed' in the 'tags' list within 'langsmith_extra'. That way, they can easily filter their traces in LangSmith based on these tags.

But wait, the context doesn't explicitly state that this is the method for marking success or failure. So I should let the user know that while I don't have specific informati

In [30]:

question = "Can I group traces by a custom field?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)


<think>
Okay, so the user is asking if they can group traces by a custom field in LangSmith. Let me think about what I know from the context provided.

The context talks about adding metadata and tags to LangChain traces using the 'langsmith_extra' parameter. It mentions that metadata can be attached with a dictionary, like having a 'metadata' key. Also, tags can be added as a list under the 'tags' key within the same parameter. These tags help in categorizing and filtering traces in LangSmith.

So, if the user wants to group traces by a custom field, they have two options. They can either use the 'metadata' field to include a custom key-value pair or add specific tags. Both methods allow for categorization. The user would need to structure their 'langsmith_extra' parameter accordingly when setting up their traces. This way, they can later use LangSmith's features to group based on these additions.

I don't see any other methods mentioned in the context, so I can't suggest anything els

In [29]:

from langsmith import traceable
from groq import Groq
from typing import List
import nest_asyncio
from utils_hf_embeddings import get_vector_db_retriever_hf

groq_client = Groq(api_key="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")
nest_asyncio.apply()

retriever = get_vector_db_retriever_hf()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the latest question in the conversation.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise.
    """
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_groq(messages)

@traceable(run_type="llm")
def call_groq(
    messages: List[dict], model: str = "deepseek-r1-distill-llama-70b", temperature: float = 0.0
) -> str:
    return groq_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

Creating new vector store at ./sklearn_vectorstore_hf.
Vector store created and persisted.


In [28]:

import uuid
thread_id = uuid.uuid4()

In [27]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path="../../.env", override=True)



False